In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sms-spam-collection-dataset/spam.csv


Part 1: Deriving Bayes theorem from conditional probability

**Conditional probability**

I covered conditional probability in more depth here.
Conditional probability tells us the probability of an event occurring, given another event.

**P(A|B) = P(A ∩ B) / P(B) is the probability A occurs, in cases where we knowB occurs. It’s calculated as the probability that both A and B occur, divided by the probability that B occurs.
But what if we wanted to find the reverse, the probability of B, in cases where A occurs?**

**sometimes it’s easier to use Bayes theorem.**

**Deriving Bayes theorem**

**We start with the formula for conditional probability which can be written either, “A given B” or “B given A”.**

**We’ll start with the 1st formula, P(A|B)= P(A∩B) / P(B).
Multiple both sides by P(B). This will cancel out the P(B) denominator on the right, leaving us with below.**

**What we can now see (more easily if we swapped the left and right sides) is that P(A∩B)= P(A|B) * P(B) . We’ll plug this back into our 2nd original formula**

![img](http://https://miro.medium.com/max/2310/1*UTDk1rC_gv90NNSyeOe4eA.png)

 Part 2: Predicting if an SMS message is spam

Bayesian inference has a long history in spam detection. We’ll get into the basics here with some real data.

![img](http://https://miro.medium.com/max/2738/1*vmYmcPTWQlJLLaY9DAXwGg.png)

**In our case, the probability an SMS is spam, given some word, is equal to the probability of the word, given it is in a spam SMS, multiplied by the probability of spam, all divided by the probability of the word.**

In [2]:
#inspect tje dataframe
dir = '../input/sms-spam-collection-dataset/spam.csv'
import pandas as pd
df = pd.read_csv(dir, encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


The columns in the original CSV don’t make sense. So we’ll move the useful information into 2 new columns, one of which is a boolean indicating if the SMS is spam.
FYI, “ham” means “not spam”.

In [3]:
import pandas as pd
df['sms'] = df['v2']
df['spam'] = np.where(df['v1'] == 'spam', 1, 0)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,sms,spam
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,"Nah I don't think he goes to usf, he lives aro...",0


now drop the old columns

In [4]:
df = df[['sms', 'spam']]
df.head()

,sms,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


check the no fo records

In [5]:
len(df)

5572

That’s a lot. Let’s work with a sample of 25% of the original data.

In [6]:
sample_df = df.sample(frac=0.25)
len(sample_df)


1393

Now split the data into 2 separate dataframes, one for spam and one for ham.

In [7]:
spam_df = sample_df.loc[df['spam'] == 1]
ham_df = sample_df.loc[df['spam'] == 0]

print(len(spam_df))
print(len(ham_df))

182
1211


We’ll use sklearn’s TFIDF vectorizer to eyeball some words important in the spam messages, and pick one to plug into our formula.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_spam = TfidfVectorizer(stop_words ='english',
max_features=30)

vectorizer_spam.fit(spam_df['sms'])
vectorizer_spam.vocabulary_

{'claim': 5,
 'new': 13,
 'reply': 16,
 '18': 2,
 'www': 29,
 'send': 17,
 'free': 8,
 'text': 20,
 'ur': 24,
 'guaranteed': 9,
 'txt': 22,
 'urgent': 25,
 '50': 3,
 'nokia': 14,
 'prize': 15,
 '150p': 0,
 'won': 28,
 'just': 10,
 'mobile': 11,
 'customer': 7,
 'cash': 4,
 'week': 26,
 'stop': 19,
 'msg': 12,
 '16': 1,
 'tone': 21,
 'uk': 23,
 'service': 18,
 'win': 27,
 'contact': 6}

We need to pick a word to use in our formula so I’m going to choose the word “win”, though it would be interesting to try this for other words as well.

P(W|S) = probability of the word “win” being in a spam message

P(S) = probability of a spam message overall

P(W) = probability of the word “win” in a message overall

In [9]:
word = 'win'

Calculate P(W|S)

In [10]:
word = 'win'
spam_count  = 0
spam_with_word_count = 0

for idx,row in spam_df.iterrows():
    spam_count += 1
    
    if word in row.sms:
        spam_with_word_count += 1

probability_of_word_given_spam = spam_count / spam_with_word_count
print(probability_of_word_given_spam)


10.705882352941176


Calculate P(S)

In [11]:
probability_of_spam = len(spam_df) / (len(sample_df))
print(probability_of_spam)


0.1306532663316583


Calculate P(W)

In [12]:
sms_count = 0
word_in_sms_count = 0
for idx,row in sample_df.iterrows():
    sms_count += 1
    
    if word in row.sms:
        word_in_sms_count += 1
probability_of_word = word_in_sms_count / sms_count
print(probability_of_word)

0.02297200287150036


Now putting it all together

In [13]:
(probability_of_word_given_spam * probability_of_spam) / probability_of_word

60.88970588235294

Boom. What this tells us is that if an SMS contains the word “win”, there is a 58% probability that the message is spam.**